In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import shutil
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import load_model

In [2]:
path  = "C:\\Users\\16395\\Desktop\\Sorghom\\"


In [3]:







train_df = pd.read_csv(path +'train_cultivar_mapping.csv')
trpath = "C:\\Users\\16395\\Desktop\\Sorghom\\"
train_df.dropna()

,image,cultivar
0,2017-06-16__12-24-20-930.png,PI_257599
1,2017-06-02__16-48-57-866.png,PI_154987
2,2017-06-12__13-18-07-707.png,PI_92270
3,2017-06-22__13-18-06-841.png,PI_152651
4,2017-06-26__12-56-48-642.png,PI_176766
...,...,...
22189,2017-06-16__12-27-16-266.png,PI_170787
22190,2017-06-28__11-19-57-454.png,PI_156393
22191,2017-06-28__10-20-32-417.png,PI_152923
22192,2017-06-28__12-47-02-714.png,PI_257599


In [4]:
IMG_SIZE = 256
BATCH_SIZE = 8

In [5]:
train_df['cultivar']=train_df['cultivar'].astype(str)

In [6]:
train_df

,image,cultivar
0,2017-06-16__12-24-20-930.png,PI_257599
1,2017-06-02__16-48-57-866.png,PI_154987
2,2017-06-12__13-18-07-707.png,PI_92270
3,2017-06-22__13-18-06-841.png,PI_152651
4,2017-06-26__12-56-48-642.png,PI_176766
...,...,...
22189,2017-06-16__12-27-16-266.png,PI_170787
22190,2017-06-28__11-19-57-454.png,PI_156393
22191,2017-06-28__10-20-32-417.png,PI_152923
22192,2017-06-28__12-47-02-714.png,PI_257599


In [7]:
# Set paths and split ratio
data_path = "C:\\Users\\16395\\Desktop\\Sorghom\\"
csv_path =  "C:\\Users\\16395\\Desktop\\Sorghom\\train_cultivar_mapping.csv"
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


In [11]:
train_df['cultivar']=train_df['cultivar'].astype(str)
val_df['cultivar']=val_df['cultivar'].astype(str)

In [9]:
train_df.count()
data_path = "C:\\Users\\16395\\Desktop\\Sorghom\\train_images"
csv_path =  "C:\\Users\\16395\\Desktop\\Sorghom\\train_cultivar_mapping.csv"
split_ratio = 0.2

# Load csv file
df = pd.read_csv(csv_path)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=split_ratio, random_state=42)


In [10]:
val_df.count()

image       4439
cultivar    4439
dtype: int64

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=path + 'train_images',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Found 17754 validated image filenames belonging to 100 classes.


C:\Users\16395\anaconda3\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 4439 validated image filenames belonging to 100 classes.


In [13]:
base_model = keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [14]:
base_model.layers

In [15]:
print(len(base_model.layers))

475


In [16]:


inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
output = base_model(inputs)
output = GlobalAveragePooling2D()(output)
output = Dense(100, activation='softmax')(output)

model = keras.Model(inputs, output)

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, 8, 8, 1792)       17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 100)               179300    
                                                                 
Total params: 17,853,123
Trainable params: 17,727,916
Non-trainable params: 125,207
_________________________________________________________________


In [17]:
history = model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
2220/2220 [==============================] - 1596s 695ms/step - loss: 3.4527 - accuracy: 0.1778 - val_loss: 2.4733 - val_accuracy: 0.3773
Epoch 2/10
2220/2220 [==============================] - 1175s 529ms/step - loss: 2.1582 - accuracy: 0.4194 - val_loss: 1.7817 - val_accuracy: 0.5373
Epoch 3/10
2220/2220 [==============================] - 11282s 5s/step - loss: 1.6245 - accuracy: 0.5454 - val_loss: 1.3502 - val_accuracy: 0.6321
Epoch 4/10
2220/2220 [==============================] - 960s 433ms/step - loss: 1.3020 - accuracy: 0.6265 - val_loss: 1.7184 - val_accuracy: 0.5983
Epoch 5/10
2220/2220 [==============================] - 1080s 486ms/step - loss: 1.0929 - accuracy: 0.6807 - val_loss: 1.2949 - val_accuracy: 0.6625
Epoch 6/10
2220/2220 [==============================] - 11652s 5s/step - loss: 0.9360 - accuracy: 0.7243 - val_loss: 1.1589 - val_accuracy: 0.6970
Epoch 7/10
2220/2220 [==============================] - 1008s 454ms/step - loss: 0.8279 - accuracy: 0.7499 - va

In [18]:

model.fit(train_generator, validation_data=valid_generator, epochs=5)

Epoch 1/5
2220/2220 [==============================] - 1320s 595ms/step - loss: 0.5482 - accuracy: 0.8327 - val_loss: 0.6282 - val_accuracy: 0.8340
Epoch 2/5
2220/2220 [==============================] - 985s 443ms/step - loss: 0.5121 - accuracy: 0.8436 - val_loss: 0.6781 - val_accuracy: 0.8123
Epoch 3/5
2220/2220 [==============================] - 1548s 697ms/step - loss: 0.4847 - accuracy: 0.8474 - val_loss: 0.6789 - val_accuracy: 0.8202
Epoch 4/5
2220/2220 [==============================] - 1762s 794ms/step - loss: 0.4389 - accuracy: 0.8644 - val_loss: 0.9363 - val_accuracy: 0.7747
Epoch 5/5
2220/2220 [==============================] - 1477s 665ms/step - loss: 0.4394 - accuracy: 0.8661 - val_loss: 0.7982 - val_accuracy: 0.8011


In [25]:
model.save("g:/effiecient.h5")

In [48]:
loaded_model = load_model("g:/effiecient.
                          h5")